## Bagging 
In Bagging we take same models and provide different samples of dataset to the models. 

In [2]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Using single model

In [7]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

In [9]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92        43
           1       0.94      0.96      0.95        71

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



### Bagging using Dtree

In [12]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [13]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        43
           1       0.96      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



Rows which base model got

In [16]:
print(bag.estimators_samples_[0].shape)
#this is a numpy list of rows pf datatset provided to each model

(113,)


In [17]:
print(bag.estimators_features_[0].shape)


(30,)


### Bagging using SVM

In [18]:
bag_svc = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.4,
    bootstrap=True,
    random_state=42
)

In [19]:
bag_svc.fit(X_train, y_train)
y_pred = bag_svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.84      0.91        43
           1       0.91      1.00      0.95        71

    accuracy                           0.94       114
   macro avg       0.96      0.92      0.93       114
weighted avg       0.94      0.94      0.94       114



## Pasting
Here we give dataset with row sampling but without replacement to multiple models

In [22]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.4,
    bootstrap=False,
    random_state=42
)

bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        43
           1       0.96      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



## Random Subspaces
Here we do column sampling

In [24]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.4,
    max_features=0.3,
    bootstrap=False,
    random_state=42
)

bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [25]:
print(bag.estimators_features_[0].shape)

(9,)


## Random Patches
Column sampling with replacement

In [26]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.4,
    max_features=0.3,
    bootstrap=True,
    random_state=42
)

bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



## OOB Score
It is a performance matrics to evaluate the bagging model. Statistically there are 67% data points whihc are provided to the model. It can be that 37% data points are never seen. Thus we use those 37% points to test the models performance

In [27]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.4,
    max_features=0.3,
    bootstrap=True,
    random_state=42,
    oob_score=True
)

bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



# Regression using Bagging
Here we provide random data subset to the models. Then when model gives the output, we aggregate by doing mean of the result of each model

In [28]:
from sklearn import datasets

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import BaggingRegressor

In [31]:
diab = datasets.load_diabetes()

In [37]:
X = diab.data
y = diab.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.shape, y.shape)

(442, 10) (442,)


In [33]:
lr = LinearRegression()
kn = KNeighborsRegressor()
dtree = DecisionTreeRegressor()

## Individual Model prediciton

In [34]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.4526027629719195

In [35]:
kn.fit(X_train, y_train)
y_pred = kn.predict(X_test)
r2_score(y_test, y_pred)

0.43016439526042805

In [36]:
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
r2_score(y_test, y_pred)

0.030857709429342295

# Using bagging on each of these models

In [44]:
bag_lr = BaggingRegressor(
    estimator=LinearRegression(),
    n_estimators=100,
    max_samples=0.2,
    bootstrap=True,
    random_state=42
)
bag_lr.fit(X_train, y_train)
y_pred = bag_lr.predict(X_test)
r2_score(y_test, y_pred)

0.4513991139269208

In [45]:
bag_dtree = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=0.2,
    bootstrap=True,
    random_state=42,
    max_features=0.3,
    oob_score=True
)
bag_dtree.fit(X_train, y_train)
y_pred = bag_dtree.predict(X_test)
r2_score(y_test, y_pred)

0.33518219423902995

In [46]:
bag_kn = BaggingRegressor(
    estimator=KNeighborsRegressor(),
    n_estimators=100,
    max_samples=0.2,
    bootstrap=True,
    random_state=42,
    max_features=0.3,
    oob_score=True
)
bag_kn.fit(X_train, y_train)
y_pred = bag_kn.predict(X_test)
r2_score(y_test, y_pred)

0.35139851014362355

# Instead of playing with random values it is better to use any CV tool.